In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.font_manager as fm
import graphviz

# 📌 데이터 다운로드 및 로드
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Pregnancies	임신 횟수	정수 (int)
# Glucose	혈당 수치 (mg/dL)	연속형 (float)
# BloodPressure	혈압 (mmHg)	연속형 (float)
# SkinThickness	피부 두께 (mm)	연속형 (float)
# Insulin	인슐린 수치 (혈청 인슐린, μU/ml)	연속형 (float)
# BMI	체질량지수(Body Mass Index)	연속형 (float)
# DiabetesPedigreeFunction	당뇨병 유전 계수 (가족력 반영 지수)	연속형 (float)
# Age	나이 (세)	정수 (int)
# Outcome	당뇨병 여부 (0: 없음, 1: 있음)	이진 분류 (int)

# 데이터셋 컬럼 이름 정의
columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
           "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]

df = pd.read_csv(url, names=columns)

# 데이터 확인
print(df.head())
print(df.info())


# 폰트 경로 설정
# font_path = "C:/Windows/Fonts/NanumGothic.ttf"
font_path= "C:/Windows/Fonts/malgun.ttf"  # '맑은 고딕' 폰트 경로
font_prop = fm.FontProperties(fname=font_path)

# Matplotlib 기본 폰트 설정
plt.rc('font', family=font_prop.get_name())

# 음수 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False
# 클래스 분포 확인
sns.countplot(x=df["Outcome"])
plt.title("당뇨병 발생 여부 분포", fontproperties=font_prop)
plt.show()


In [ ]:
# 🎯 독립 변수(X)와 종속 변수(y) 정의
X = df.drop(columns=["Outcome"])  # 8개의 건강 지표
y = df["Outcome"]  # 0(당뇨 아님), 1(당뇨병)

# 📌 데이터 분할 (훈련 80%, 테스트 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📊 데이터 정규화 (스케일링)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# 🌳 결정 트리 모델 학습
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train_scaled, y_train)

# 🎯 예측 수행
y_pred = model.predict(X_test_scaled)

# 📌 모델 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"모델 정확도: {accuracy:.2f}")

# 🔥 혼동 행렬 시각화
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("예측값")
plt.ylabel("실제값")
plt.title("혼동 행렬")
plt.show()

# 📊 분류 보고서 출력
print(classification_report(y_test, y_pred))


In [ ]:
# 샘플 데이터 3개 생성 (기존 데이터의 평균 및 범위를 참고)
sample_data = pd.DataFrame(
    [
        [2, 120, 70, 30, 80, 25.0, 0.5, 35],  # 예제 1: 정상 범위
        [5, 180, 88, 35, 120, 30.0, 0.7, 50], # 예제 2: 당뇨 위험
        [0, 95, 60, 25, 60, 22.0, 0.3, 21]   # 예제 3: 건강한 젊은 사람
    ],
    columns=X.columns  # 원래 학습 데이터와 동일한 열 이름 사용
)

# 📌 스케일링 적용 (훈련 데이터와 같은 방식으로 변환)
sample_data_scaled = scaler.transform(sample_data)

# 🎯 예측 수행
sample_predictions = model.predict(sample_data_scaled)
sample_probabilities = model.predict_proba(sample_data_scaled)[:, 1]  # 당뇨병(1)일 확률

# 🔍 결과 출력
for i, (pred, prob) in enumerate(zip(sample_predictions, sample_probabilities)):
    print(f"샘플 {i+1}: 예측 결과 = {'당뇨병 있음(1)' if pred == 1 else '당뇨병 없음(0)'} (확률: {prob:.2f})")


In [ ]:
# 📌 결정 트리 모델을 Graphviz 형식으로 변환
from sklearn.tree import export_graphviz

dot_data = export_graphviz(
    model,                      # 🎯 결정 트리 모델
    out_file=None,              # 파일 저장 안 함 (None이면 문자열로 반환)
    feature_names=X.columns,    # 📌 특성(Feature) 이름 지정
    class_names=["No Diabetes", "Diabetes"],  # 📌 클래스 이름 지정
    filled=True,                # 📌 색상 채우기 (Gini 계수 기반)
    rounded=True,               # 📌 둥근 모서리 적용
    special_characters=True      # 📌 특수문자 지원 (텍스트 오류 방지)
)


# Graphviz 객체 생성 및 출력
graph = graphviz.Source(dot_data)
graph.render("decision_tree")  # PDF 파일로 저장 가능
graph.view()  # 생성된 그래프 열기